## Imports and libraries version adaptation

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip install -q 'scipy<=1.2.1'  # scipy.misc.imread is deprecated in later versions

     |████████████████████████████████| 24.8 MB 56.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lucid 0.3.10 requires umap-learn, which is not installed.
lucid 0.3.10 requires numpy<=1.19, but you have numpy 1.19.5 which is incompatible.
kapre 0.3.6 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import os, cv2, shutil, scipy.misc, warnings
from PIL import Image
import matplotlib.pyplot as plt
from math import sqrt 
import numpy as np
from scipy.spatial import distance
from numpy.core.fromnumeric import shape
from google.colab import drive

warnings.filterwarnings("ignore")
drive.mount("/content/drive")

Mounted at /content/drive


## Network setting

In [ ]:
!git clone https://github.com/titu1994/Neural-Style-Transfer.git

Cloning into 'Neural-Style-Transfer'...
remote: Enumerating objects: 1411, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 1411 (delta 5), reused 6 (delta 1), pack-reused 1393
Receiving objects: 100% (1411/1411), 68.18 MiB | 22.69 MiB/s, done.
Resolving deltas: 100% (824/824), done.


In [ ]:
nst_path = "Neural-Style-Transfer"

## Network type
NETWORK = 'Network' + '.py'

## Network parameters
IMAGE_SIZE = 256

# Loss Weights
CONTENT_WEIGHT = 0.025
STYLE_WEIGHT = 1.0
STYLE_SCALE = 1.0
TOTAL_VARIATION_WEIGHT = 8.5e-5
CONTENT_LOSS_TYPE = 0

# Training arguments
NUM_ITERATIONS = 10
MODEL = 'vgg19'
RESCALE_IMAGE = 'false'
MAINTAIN_ASPECT_RATIO = 'false'  # Set to false if OOM occurs

# Transfer Arguments
CONTENT_LAYER = 'conv' + '5_2'  # only change the number 5_2 to something in a similar format
INITIALIZATION_IMAGE = 'content'
POOLING_TYPE = 'max'

# Extra arguments
PRESERVE_COLOR = 'false'
MIN_IMPROVEMENT = 0.0

## Helper methods definition

In [ ]:
def show_img(img, title=''):
  if type(img) == str:
    img = plt.imread(img)

  plt.axis('off')
  plt.title(title)
  plt.imshow(img)
  plt.show()

def transfer_style(result_dir, content_fn, style_fn):
  RESULT_PREFIX = os.path.join(result_dir, "gen")
  FINAL_IMAGE_PATH = RESULT_PREFIX + "_at_iteration_%d.png" % (NUM_ITERATIONS)

  run_nst_script(RESULT_PREFIX, content_fn, style_fn)

  return FINAL_IMAGE_PATH

def run_nst_script(result_prefix, content_fn, style_fn):
  %%shell
  !python {nst_path}/{NETWORK} {content_fn} {style_fn} {result_prefix} \
    --image_size {IMAGE_SIZE} --content_weight {CONTENT_WEIGHT} --style_weight \
    {STYLE_WEIGHT} --style_scale {STYLE_SCALE} --total_variation_weight \
    {TOTAL_VARIATION_WEIGHT} --content_loss_type {CONTENT_LOSS_TYPE} --num_iter \
    {NUM_ITERATIONS} --model {MODEL} --rescale_image {RESCALE_IMAGE} \
    --maintain_aspect_ratio {MAINTAIN_ASPECT_RATIO} --content_layer {CONTENT_LAYER} \
    --init_image {INITIALIZATION_IMAGE} --pool_type {POOLING_TYPE} --preserve_color \
    {PRESERVE_COLOR} --min_improvement {MIN_IMPROVEMENT}

In [ ]:
def transfer_color(path, content_fn):
  COLOR_TRANSFER = 'color_transfer.py'
  COLOR_FINAL_IMAGE_PATH = path + '_%s_color.png'

  # Optional - Use Histogram matching (0 for no, 1 for yes)
  HISTOGRAM_MATCH = 0

  if HISTOGRAM_MATCH == 0:
    COLOR_FINAL_IMAGE_PATH = COLOR_FINAL_IMAGE_PATH % ('original')
  else:
    COLOR_FINAL_IMAGE_PATH = COLOR_FINAL_IMAGE_PATH % ('histogram')
    
  %%shell
  !python {nst_path}/{COLOR_TRANSFER} {content_fn} {path} --hist_match {HISTOGRAM_MATCH}

# Style transfer

In [ ]:
base_dir = 'drive/MyDrive/_Van_Gogh/results/'
folders = os.listdir(base_dir)

for f in folders:
  save_dir = os.path.join(base_dir, f, 'NST')
  if not os.path.exists(save_dir):
    os.mkdir(save_dir)
  else: # if directory exists, it means the image has already been processed
    continue
  
  num = f[f.index('_')+1:]

  content_fn = os.path.join(base_dir, f, num + '.jpg')
  style_fn = os.path.join(base_dir, f, num + '_style.jpg')

  print("Processing image " + num)
  print()

  # transfer the style
  transferred_img = transfer_style(save_dir, content_fn, style_fn)

  # transfer the color
  transfer_color(transferred_img, content_fn)

  print()
  print("Done!")
  print()
  print()

Output streaming troncato alle ultime 5000 righe.
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  generated_image = imread(args.generated_image, mode="RGB")
Neural-Style-Transfer/color_transfer.py:119: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  content_image = imread(args.content_image, mode=mode)
Neural-Style-Transfer/color_transfer.py:120: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  content_image = imresize(content_image, (img_width, img_height), interp='bicubic')
Neural-Style-Transfer/color_transfer.py:58: DeprecationWarning:     `toimage` is deprecated!
    `toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use Pillow's ``Image.fro

# Results visualization

In [ ]:
for f in folders:
  num = f[f.index('_')+1:]
  save_dir = os.path.join(base_dir, f, 'NST')
  
  content_img = Image.open(os.path.join(base_dir, f, num + '.jpg'))
  style_img = Image.open(os.path.join(base_dir, f, num + '_style.jpg'))
  result_no_color = Image.open(os.path.join(save_dir, 'gen_at_iteration_%d_original_color.png' % (NUM_ITERATIONS)))
  result_color = Image.open(os.path.join(save_dir, 'gen_at_iteration_%d.png' % (NUM_ITERATIONS)))

  print("Content:")
  show_img(content_img)
  print("Style:")
  show_img(style_img)
  print("Result (no color transfer):")
  show_img(result_no_color)
  print("Result (with color transfer):")
  show_img(result_color)
  print()

Output hidden; open in https://colab.research.google.com to view.